# Episode 67. Tensorflow 2 GradientTape

## Import Libraries

In [ ]:
import tensorflow as tf
print(tf.__version__) # Should be >= 2.0.0

2.8.0


## Evaluate Derivatives

In [ ]:
x = tf.Variable(3.0, trainable=True)

In [ ]:
with tf.GradientTape() as tape:
    y = x**2

dy_dx = tape.gradient(y,x)
print(dy_dx)

del tape, x

tf.Tensor(6.0, shape=(), dtype=float32)


## Linear Regression

In [ ]:
# import libraries
import random
import numpy as np

# Training data
x_train = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_train = np.asarray([i*2+1 for i in x_train])+np.random.normal(0,0.1,11) # y = 2x+1+noise
print(y_train)

[ 1.15923793  3.16866911  4.8991129   7.12876485  8.93635803 11.03282465
 12.9034784  14.94350448 17.22664561 19.06659529 21.09608088]


In [ ]:
# Regression parameters
a = tf.Variable(random.random(), trainable=True)
b = tf.Variable(random.random(), trainable=True)

In [ ]:
# Loss function
def loss(y, y_pred):
    return tf.abs(y - y_pred)

In [ ]:
# minimize least absolute error - run dataset once
def onestep(x, y):
    with tf.GradientTape(persistent=True) as tape:
        # Make prediction
        y_pred = a * x + b
        # Calculate loss
        total_loss = loss(y, y_pred)
    
    # Calculate gradients
    a_gradients, b_gradients = tape.gradient(total_loss, (a, b))

    # Update variables
    a.assign_sub(a_gradients * 0.001)
    b.assign_sub(b_gradients * 0.001)

In [ ]:
# Training loop - repeat dataset 10000 times
for _ in range(10000):
    onestep(x_train, y_train)

print(f'y ≈ {a.numpy()}x + {b.numpy()}')

del a, b, x_train, y_train, onestep, loss

y ≈ 1.9954780340194702x + 1.1574337482452393


## MNIST Classification

In [ ]:
# import libraries
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import random
import math
%matplotlib inline

In [ ]:
# Load and pre-process training data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255).reshape((60000, 784))
y_train = tf.keras.utils.to_categorical(y_train, 10)
x_test = (x_test / 255).reshape((10000, 784))
y_test = tf.keras.utils.to_categorical(y_test, 10)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# set parameters
batch_size = 128
epochs = 25

In [ ]:
# Build model
model = Sequential()
model.add(Dense(512,  activation='relu', input_shape=(784,)))
model.add(Dense(128,  activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# optimizer
opt = Adam(lr=0.001)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# optimization for a single step
def onestep(x, y):
    with tf.GradientTape() as tape:
        # Make prediction
        y_pred = model(x)
        # Calculate loss
        model_loss = tf.keras.losses.categorical_crossentropy(y, y_pred)
    
    # Calculate gradients
    model_gradients = tape.gradient(model_loss, model.trainable_variables)
    # Update model
    opt.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
# Training loop
bat_per_epoch = math.floor(len(x_train) / batch_size)
for epoch in range(epochs):
    print('=', end='')
    for i in range(bat_per_epoch):
        n = i*batch_size
        onestep(x_train[n:n+batch_size], y_train[n:n+batch_size])

In [ ]:
# Calculate accuracy
model.compile(optimizer=opt, loss=tf.losses.categorical_crossentropy, metrics=['acc']) # Compile just for evaluation
print('\nAccuracy:', model.evaluate(x_test, y_test, verbose=0)[1])


Accuracy: 0.9828000068664551
